---
## Dependencias 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import ast
import numpy as np
import math
import seaborn as sns

## Carga archivos

In [ ]:
# Cargar los datos preprocesados
df_libro = pd.read_csv("../data/output/libro_cev_preprocesado.csv", sep="|", encoding="utf-8")
df_testimonios = pd.read_csv("../data/output/testimonios_cev_preprocesado.csv", sep="|", encoding="utf-8")

In [ ]:
# Convertir las cadenas de listas de vuelta a listas
df_libro["contenido_preprocesado"] = df_libro["contenido_preprocesado"].apply(ast.literal_eval)
df_testimonios["contenido_preprocesado"] = df_testimonios["contenido_preprocesado"].apply(ast.literal_eval)

In [ ]:
# Convertir las listas de listas en texto plano
df_libro["contenido_texto"] = df_libro["contenido_preprocesado"].apply(lambda row: " ".join([word for phrase in row for word in phrase]))
df_testimonios["contenido_texto"] = df_testimonios["contenido_preprocesado"].apply(lambda row: " ".join([word for phrase in row for word in phrase]))

## Crear matriz Tfidf

In [ ]:
tfidf = TfidfVectorizer()
tfs = tfidf.fit_transform(df_libro["contenido_texto"])

In [ ]:
print(len(tfidf.get_feature_names_out()))
print(tfidf.get_feature_names_out())

In [ ]:
secc_num, feature_num = tfs.shape
feature_names = tfidf.get_feature_names_out()
print("# secciones: %d, n_features: %d" % tfs.shape)

In [ ]:
print("###### Calculo de Feature Names ######")
for x in range(0, feature_num):
    print(" # ", x ," - ",feature_names[x], " \t - ", [tfs[n,x] for n in range(0, secc_num)])

## Implementacion metricas de reelevancia

In [ ]:
def as_doc(text):
    return [text]
    
response = tfidf.transform(as_doc(df_testimonios["contenido_texto"].iloc[0]))
print('response:', response)

In [ ]:
cosine_similarity_response =  cosine_similarity(response, tfs)

In [ ]:
print("n_question: %d, n_features: %d" % response.shape)
print("cosine_similarity ", cosine_similarity_response)

In [ ]:
def compute_similarities(doc):
    # Transform single doc into TF-IDF using the trained vectorizer
    response = tfidf.transform([doc])
    # Cosine similarity vs. all docs in libro
    sims = cosine_similarity(response, tfs)[0]  # [0] to flatten 2D → 1D array
    return sims

# Apply row by row
df_testimonios["cosine_similarities"] = df_testimonios["contenido_texto"].apply(compute_similarities)

In [ ]:
df_testimonios["cosine_similarities"].iloc[0]

In [ ]:
top_n = 5
df_testimonios["top_matches"] = df_testimonios["cosine_similarities"].apply(
    lambda sims: sorted(enumerate(sims), key=lambda x: x[1], reverse=True)[:top_n]
)

In [ ]:
df_testimonios["top_matches"].iloc[0]

----
## Metrics

### Rocchio
El modelo Rocchio ajusta el vector de consulta en el espacio vectorial (TF-IDF) usando documentos relevantes y no relevantes que el usuario marca.

Interpretación:

* Empiezas con la consulta original.
* Le agregas “peso” de los documentos relevantes.
* Le restas “peso” de los documentos no relevantes.
* Resultado: un query vector modificado, que refleja mejor la intención del usuario.

Es un método de realimentación de relevancia (relevance feedback) que ajusta el vector de consulta en el espacio vectorial.

1.  Tienes un espacio de documentos representados con TF-IDF.

2.  Tomas tu consulta original y la conviertes en vector (q₀).

3.  Rocchio la “mueve” en ese espacio, agregando peso hacia los documentos relevantes y alejándose de los no relevantes:

4.  Una vez obtienes qnuevo, necesitas comparar ese vector contra los documentos → y ahí entra la similitud coseno.

* Cosine Similarity: métrica de comparación entre vectores (cuán “cerca” están dos documentos o una consulta y un documento).

* Rocchio: técnica que modifica la consulta para que, al medir con cosine, los documentos relevantes queden más cerca y los no relevantes más lejos.

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def rocchio(query_vec, tfs, relevant_idx=None, nonrelevant_idx=None, alpha=1.0, beta=0.75, gamma=0.15):
    """
    query_vec: vector TF-IDF de la consulta (1 x n_features)
    tfs: matriz TF-IDF de todos los documentos (n_docs x n_features)
    relevant_idx: lista de índices de docs relevantes
    nonrelevant_idx: lista de índices de docs no relevantes
    """
    q0 = query_vec.toarray()[0]
    q_new = alpha * q0

    if relevant_idx:
        q_new += beta * np.mean(tfs[relevant_idx].toarray(), axis=0)

    if nonrelevant_idx:
        q_new -= gamma * np.mean(tfs[nonrelevant_idx].toarray(), axis=0)

    return q_new.reshape(1, -1)

In [ ]:
# Aplicar a todas las entrevistas
all_rocchio_scores = []
all_rocchio_top = []

for query in df_testimonios["contenido_texto"]:
    q_vec = tfidf.transform([query])

    # Nota: si no defines relevantes/no relevantes, usamos solo q0 (consulta original)
    q_new = rocchio(q_vec, tfs, relevant_idx=[], nonrelevant_idx=[])

    # Similitud coseno entre la nueva consulta y todo libro
    scores = cosine_similarity(q_new, tfs)[0]
    all_rocchio_scores.append(scores)

    # Top-5 documentos más similares
    top5 = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)[:5]
    all_rocchio_top.append(top5)

df_testimonios["rocchio"] = all_rocchio_scores
df_testimonios["rocchio_top"] = all_rocchio_top


In [ ]:
df_testimonios["rocchio"].iloc[0]

In [ ]:
df_testimonios["rocchio_top"].iloc[0]

In [ ]:
df_testimonios["rocchio_top"]

### Okapi BM25
BM25 es un modelo probabilístico basado en frecuencia de términos (no en TF-IDF).
Para cada término de la consulta mide qué tan representativo es en el documento, ajustando por:

Longitud del documento (∣𝑑∣): documentos más largos no deben dar ventaja injusta.

Frecuencia del término en el documento (𝑓(𝑡,𝑑).

IDF (inversa de frecuencia de documento): términos poco frecuentes en la colección tienen más peso.

Interpretación:

* Si un término aparece mucho en un documento, la relevancia aumenta pero se
satura (no crece infinito).
* Documentos largos se penalizan ligeramente.
* Palabras raras en la colección (alto IDF) pesan más.

In [ ]:
import math

def compute_idf(corpus_tokens):
    """
    Calcula los valores IDF para cada término del corpus.
    corpus_tokens: lista de listas de palabras por documento
    """
    N = len(corpus_tokens)
    df = {}
    for doc in corpus_tokens:
        for term in set(doc):  # contamos solo 1 vez por doc
            df[term] = df.get(term, 0) + 1
    idf = {term: math.log((N - df[term] + 0.5) / (df[term] + 0.5) + 1) for term in df}
    return idf

def bm25_score(query_tokens, doc_tokens, idf, avgdl, k1=1.5, b=0.75):
    """
    Calcula BM25 entre una consulta y un documento.
    """
    score = 0.0
    doc_len = len(doc_tokens)

    # frecuencias de términos en el documento
    freqs = {}
    for term in doc_tokens:
        freqs[term] = freqs.get(term, 0) + 1

    for term in query_tokens:
        if term not in idf:
            continue
        f = freqs.get(term, 0)
        if f == 0:
            continue
        denom = f + k1 * (1 - b + b * doc_len / avgdl)
        score += idf[term] * (f * (k1 + 1)) / denom
    return score

def bm25_pipeline(corpus_tokens, query_tokens, k1=1.5, b=0.75):
    """
    Calcula BM25 de una consulta contra todo el corpus.
    """
    idf = compute_idf(corpus_tokens)
    avgdl = sum(len(doc) for doc in corpus_tokens) / len(corpus_tokens)

    scores = [bm25_score(query_tokens, doc, idf, avgdl, k1, b) for doc in corpus_tokens]
    return scores

In [ ]:
# Para LIBRO
df_libro["contenido_tokens"] = df_libro["contenido_preprocesado"].apply(
    lambda row: [word for phrase in row for word in phrase]
)

# Para ENTREVISTAS
df_testimonios["contenido_tokens"] = df_testimonios["contenido_preprocesado"].apply(
    lambda row: [word for phrase in row for word in phrase]
)

all_bm25_scores = []
all_bm25_top = []

idf = compute_idf(df_libro["contenido_tokens"])
avgdl = sum(len(doc) for doc in df_libro["contenido_tokens"]) / len(df_libro["contenido_tokens"])

for query in df_testimonios["contenido_tokens"]:
    scores = [bm25_score(query, doc, idf, avgdl) for doc in df_libro["contenido_tokens"]]
    all_bm25_scores.append(scores)

    top5 = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)[:5]
    all_bm25_top.append(top5)

df_testimonios["bm25"] = all_bm25_scores
df_testimonios["bm25_top"] = all_bm25_top

In [ ]:
df_testimonios["bm25_top"].iloc[0]

## 5 Comparacion corpus

Diferencias en escala

1.  TF-IDF + Cosine / Rocchio

* El cosine_similarity siempre devuelve valores en el rango [0, 1].

* 1 = vectores idénticos, 0 = no comparten nada.

* Es una métrica normalizada → siempre acotada.

2.  BM25

* Los valores no están normalizados.

* Cada término contribuye al score de manera acumulativa.

* Cuantos más términos de la consulta aparecen en el documento (y con alta frecuencia), más grande el score.

* Escala depende de:

  *  Tamaño de la consulta (más palabras → score más alto).

  * Frecuencias de los términos.

  * Valores de los hiperparámetros
  
* No tiene un límite superior fijo: 5, 10, 20 o más son posibles.

In [ ]:
def extract_doc_ids(column):
    return [doc_id for row in df_testimonios[column] for doc_id, score in row]

# Listas con los doc_ids que aparecen en Top-5
cosine_docs = extract_doc_ids("top_matches")
rocchio_docs = extract_doc_ids("rocchio_top")
bm25_docs   = extract_doc_ids("bm25_top")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,5))

# --- TF-IDF + Cosine ---
plt.subplot(1,3,1)
plt.hist(cosine_docs, bins=range(len(df_libro)+1), edgecolor="black")
plt.title("TF-IDF + Cosine")
plt.xlabel("Doc ID en libro")
plt.ylabel("Frecuencia en Top-5")

# --- Rocchio ---
plt.subplot(1,3,2)
plt.hist(rocchio_docs, bins=range(len(df_libro)+1), edgecolor="black", color="orange")
plt.title("Rocchio + Cosine")
plt.xlabel("Doc ID en libro")

# --- BM25 ---
plt.subplot(1,3,3)
plt.hist(bm25_docs, bins=range(len(df_libro)+1), edgecolor="black", color="green")
plt.title("BM25")
plt.xlabel("Doc ID en libro")

plt.tight_layout()
plt.show()


In [ ]:
def build_top5_matrix(df, column_name, n_docs):
    """
    Construye una matriz (entrevistas x documentos) con los scores de los top-5.
    df: DataFrame de entrevistas
    column_name: "top_matches", "rocchio_top" o "bm25_top"
    n_docs: número de documentos en libro
    """
    matrix = np.zeros((len(df), n_docs))
    for i, row in enumerate(df[column_name]):
        for doc_id, score in row:
            matrix[i, doc_id] = score
    return pd.DataFrame(matrix, index=[f"E{i}" for i in range(len(df))],
                        columns=[f"S{j}" for j in range(n_docs)])


In [ ]:
cosine_matrix = build_top5_matrix(df_testimonios, "top_matches", len(df_libro))
rocchio_matrix = build_top5_matrix(df_testimonios, "rocchio_top", len(df_libro))
bm25_matrix = build_top5_matrix(df_testimonios, "bm25_top", len(df_libro))

# Ejemplo para Cosine
plt.figure(figsize=(12,6))
sns.heatmap(cosine_matrix, cmap="YlOrRd", annot=False, cbar=True)
plt.title("Heatmap Top-5 TF-IDF + Cosine")
plt.xlabel("Secciones en libro")
plt.ylabel("Entrevistas")
plt.show()

# Construir matriz para Rocchio
bm25_matrix = build_top5_matrix(df_testimonios, "bm25_top", len(df_libro))

plt.figure(figsize=(12,6))
sns.heatmap(rocchio_matrix, cmap="YlGnBu", annot=False, cbar=True)
plt.title("Heatmap Top-5 BM25 (Entrevistas x Seccion)")
plt.xlabel("Secciones en libro")
plt.ylabel("Entrevistas")
plt.show()

# Construir matriz para BM25
bm25_matrix = build_top5_matrix(df_testimonios, "bm25_top", len(df_libro))

plt.figure(figsize=(12,6))
sns.heatmap(bm25_matrix, cmap="YlGnBu", annot=False, cbar=True)
plt.title("Heatmap Top-5 BM25 (Entrevistas x Seccion)")
plt.xlabel("Secciones en libro")
plt.ylabel("Entrevistas")
plt.show()


In [ ]:
from sklearn.preprocessing import minmax_scale

# Normalizamos la matriz BM25 fila por fila (cada query)
bm25_matrix_norm = bm25_matrix.apply(lambda row: minmax_scale(row), axis=1, result_type="expand")

sns.heatmap(bm25_matrix_norm, cmap="YlGnBu")
plt.title("Heatmap BM25 (normalizado 0-1 por consulta)")
plt.show()

In [ ]:
import numpy as np
import pandas as pd

def build_top5_count_matrix(df, column_name, n_docs):
    """
    Construye una matriz (entrevistas x documentos) con la frecuencia de aparición en Top-5.
    df: DataFrame de entrevistas
    column_name: "top_matches", "rocchio_top" o "bm25_top"
    n_docs: número de documentos en libro
    """
    matrix = np.zeros((len(df), n_docs))
    for i, row in enumerate(df[column_name]):
        for doc_id, _ in row:   # ignoramos score, solo doc_id
            matrix[i, doc_id] += 1
    return pd.DataFrame(matrix, index=[f"E{i}" for i in range(len(df))],
                        columns=[f"S{j}" for j in range(n_docs)])


In [ ]:
cosine_count = build_top5_count_matrix(df_testimonios, "top_matches", len(df_libro))
rocchio_count = build_top5_count_matrix(df_testimonios, "rocchio_top", len(df_libro))
bm25_count   = build_top5_count_matrix(df_testimonios, "bm25_top", len(df_libro))

In [ ]:
plt.figure(figsize=(15,12))

# --- TF-IDF + Cosine ---
plt.subplot(3,1,1)
sns.heatmap(cosine_count, cmap="Blues", cbar=True, annot=False)
plt.title("Heatmap de frecuencias en Top-5 (TF-IDF + Cosine)")
plt.xlabel("Secciones en libro")
plt.ylabel("Entrevistas")

# --- Rocchio ---
plt.subplot(3,1,2)
sns.heatmap(rocchio_count, cmap="Oranges", cbar=True, annot=False)
plt.title("Heatmap de frecuencias en Top-5 (Rocchio)")
plt.xlabel("Secciones en libro")
plt.ylabel("Entrevistas")

# --- BM25 ---
plt.subplot(3,1,3)
sns.heatmap(bm25_count, cmap="Greens", cbar=True, annot=False)
plt.title("Heatmap de frecuencias en Top-5 (BM25)")
plt.xlabel("Secciones en libro")
plt.ylabel("Entrevistas")

plt.tight_layout()
plt.show()


In [ ]:
df_libro